In [15]:
import numpy as np
import scipy as sp
from scipy.special import gamma

In [104]:
def params_to_moments(params):
    N = params[0:2]
    k = params[2:4]
    theta = params[4:]

    moments = np.zeros_like(params)
    for j in range(len(moments)):
        moments[j] = N[0]*gamma(k[0]+j)/gamma(k[0])*theta[0]**j + N[1]*gamma(k[1]+j)/gamma(k[1])*theta[1]**j
    
    return moments

def Jacobian_moments_of_params(params):
    N = np.asarray(params[0:2])
    k = np.asarray(params[2:4])
    theta = np.asarray(params[4:])
    # Jm[k,j] = dMk/dpj
    Jm = np.zeros((len(params), len(params)))
    M_k = np.zeros((len(params),2))
    dM_k = np.zeros((len(params),2))

    Jm[0,0:2] = 1
    M_k[0,:] = 0
    dM_k[0,:] = 0

    Jm[1,0:2] = k*theta
    M_k[1,:] = k
    # 2 + 1*(3)
    dM_k[1,:] = 1
    Jm[1,2:4] = N*theta*dM_k[1,:]
    Jm[1,4:]  = N*k
    for j in range(2, len(params)):
        Jm[j,0:2] = gamma(k + j)/gamma(k)*theta**j
        M_k[j,:] = M_k[j-1,:]*(k+j-1)
        dM_k[j,:] = M_k[j-1,:] + dM_k[j-1,:]*(k+j-2)
        Jm[j,2:4]  = N * theta**j * dM_k[j,:]
        Jm[j,4:] = Jm[j,0:2] * j / theta * N * (k+1)
    
    return Jm

In [105]:
params_init = [100, 1, 2, 3, 10, 100]
moments_init = params_to_moments(params_init)
J = Jacobian_moments_of_params(params_init)